In [1]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution1D, MaxPooling1D
from keras import metrics

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.utils import np_utils

import DataProcessCNN as DP

Using TensorFlow backend.


In [2]:
#for subj_n in range(1,10):
    #load the data
#    subj_filename='C:/Users/lt/Desktop/DSW2018/PAMAP2_Dataset/Protocol/subject10'+str(subj_n)+'.dat'
#    dp=DP.dataprocess(subj_filename)
#    np.save('data'+str(subj_n)+'.npy', (DP.col_sublabels,dp.data_segmented))

In [3]:
datalabels=[]
for i in range(1,10):
    feature_names,datalabelsi=np.load('data'+str(i)+'.npy')
    datalabels.append(datalabelsi)

datalabels=np.vstack(datalabels)

act_list=np.unique(datalabels[:,0,1])

In [4]:
X_all=datalabels[:,:,2:]
y_all=datalabels[:,0,1]

y_all2=np.zeros(len(y_all))
for i in range(12):
    y_all2[np.where(y_all==act_list[i])]=i

#convert to float32
X_all = X_all.astype('float32')

# convert labels to class matrix, one-hot-encoding
Y_all = np_utils.to_categorical(y_all2,12)
# split in train and test set
X_train, x_test, Y_train, y_test = train_test_split(X_all, Y_all, test_size=0.2)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train.reshape(len(X_train)*512,31))

X_trainnorm=(X_train-scaler.mean_)/scaler.scale_
x_testnorm=(x_test-scaler.mean_)/scaler.scale_

In [6]:
model = Sequential()
model.add(Convolution1D(16, 3, activation='relu', input_shape=(512,31)))
model.add(MaxPooling1D(3))
model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dense(12, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [7]:
model.fit(X_trainnorm, Y_train, validation_data=(x_testnorm,y_test), epochs=10, batch_size=16,shuffle=True)

Train on 2664 samples, validate on 666 samples
Epoch 1/10
2664/2664 [==============================] - 2s 616us/step - loss: 1.1506 - acc: 0.6175 - val_loss: 0.5463 - val_acc: 0.8168
Epoch 2/10
2664/2664 [==============================] - 1s 493us/step - loss: 0.2950 - acc: 0.9122 - val_loss: 0.2499 - val_acc: 0.9234
Epoch 3/10
2664/2664 [==============================] - 1s 493us/step - loss: 0.1311 - acc: 0.9640 - val_loss: 0.1840 - val_acc: 0.9429
Epoch 4/10
2664/2664 [==============================] - 1s 476us/step - loss: 0.0745 - acc: 0.9824 - val_loss: 0.1509 - val_acc: 0.9535
Epoch 5/10
2664/2664 [==============================] - 1s 498us/step - loss: 0.0478 - acc: 0.9869 - val_loss: 0.1366 - val_acc: 0.9610
Epoch 6/10
2664/2664 [==============================] - 1s 498us/step - loss: 0.0235 - acc: 0.9959 - val_loss: 0.1171 - val_acc: 0.9655
Epoch 7/10
2664/2664 [==============================] - 1s 498us/step - loss: 0.0168 - acc: 0.9970 - val_loss: 0.1111 - val_acc: 0.9640
E